In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import optuna

/home/carolima/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v9_median.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v9_median.csv")



In [3]:
# Separating features and target variable
X = challenge_set_updated.drop('tow', axis=1)
y = challenge_set_updated['tow']

In [4]:
# Dropping unnecessary columns
to_drop = [
    'offblock_to_arrival_duration', 'normalized_taxi_ratio', 'MALW_kg', 'wind_distance_ARR_100', 
    'average_airspeed_ARR_100', 'track_variation_ARR_100', 'is_offblock_weekend', 'Num_Engines', 
    'flown_distance_ARR_100', 'average_humidity_ARR_100', 'average_temperature_ARR_100', 
    'arrival_minute', 'track_variation_ENR', 'groundspeed_ARR_100', 'average_vertical_rate_ARR_100', 
    'taxiout_time', 'offblock_minute', 
    'average_airspeed_ENR', 'specific_energy_ENR', 'taxi_ratio',  
    'specific_energy_ARR_100', 'is_offblock_rush_hour', 'wind_distance_ENR', 'groundspeed_ENR', 
    'altitude_difference', 'average_vertical_rate_ENR', 'bearing', 'Altitude_ades'
]


X.drop(to_drop, axis=1, inplace=True)

In [5]:
cat_names = [
    'adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_hour', 'offblock_minute', 
    'offblock_day_of_week', 'offblock_month', 'offblock_week_of_year', 'offblock_season', 
    'arrival_hour', 'arrival_minute', 'is_offblock_weekend', 'is_offblock_rush_hour', 
    'flight_duration_category', 'adep_region', 'ades_region', 'same_country_flight', 
    'same_region_flight', 'flight_direction', 'is_intercontinental', 'Manufacturer', 
    'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight', 'adep_geo_cluster', 'ades_geo_cluster'
]

# Filtering categorical features that are still present after dropping columns
selected_cat_names = [x for x in cat_names if x not in to_drop]

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Handle categorical columns by Label Encoding
label_encoder = LabelEncoder()

# Specify the columns that are categorical
categorical_cols = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_season', 
                    'flight_duration_category', 'adep_region', 'ades_region', 'flight_direction', 
                    'Manufacturer', 'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight']

# Convert these categorical columns into numerical form using Label Encoding
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Now, you can proceed with training LightGBM with this processed data


In [7]:
# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical feature indices for LightGBM
cat_feature_indices = [X_train.columns.get_loc(col) for col in selected_cat_names if col in X_train.columns]

# Define the objective function for hyperparameter tuning with Optuna
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 0.2),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.0, 10.0),
        'device_type': 'gpu',  # Use GPU for training

    }

    model = lgb.LGBMRegressor(**params, n_estimators=5000)

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
        categorical_feature=cat_feature_indices
    )
    
    val_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, val_pred, squared=False)
    
    return rmse

In [8]:
# Create the Optuna study and start optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# Display the best hyperparameters found
print(f"Best trial: {study.best_trial.params}")


[I 2024-09-25 11:17:04,340] A new study created in memory with name: no-name-1d30e969-a543-46a8-a9f1-6a201312e5c8


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2606]	valid_0's rmse: 2708.85


/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-25 11:17:20,147] Trial 0 finished with value: 2708.8505588056096 and parameters: {'learning_rate': 0.06824013703023082, 'num_leaves': 568, 'max_depth': 7, 'min_data_in_leaf': 26, 'feature_fraction': 0.7645742084353451, 'bagging_fraction': 0.762039990749708, 'lambda_l1': 4.0172032540830855, 'lambda_l2': 3.1558944189389737, 'min_gain_to_split': 0.1505361144667464, 'min_child_weight': 5.673169545674453}. Best is trial 0 with value: 2708.8505588056096.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3188]	valid_0's rmse: 2710.54


/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-25 11:17:41,121] Trial 1 finished with value: 2710.539239300816 and parameters: {'learning_rate': 0.09482995200078194, 'num_leaves': 2931, 'max_depth': 7, 'min_data_in_leaf': 53, 'feature_fraction': 0.40083426098264774, 'bagging_fraction': 0.9495950196575347, 'lambda_l1': 7.796807558927977, 'lambda_l2': 9.05108001278872, 'min_gain_to_split': 0.17753972404427532, 'min_child_weight': 9.285691009651972}. Best is trial 0 with value: 2708.8505588056096.


...

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[4971]	valid_0's rmse: 2646.51


/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-25 11:50:26,788] Trial 28 finished with value: 2646.509352133502 and parameters: {'learning_rate': 0.01712847927294276, 'num_leaves': 1979, 'max_depth': 11, 'min_data_in_leaf': 65, 'feature_fraction': 0.44853227301909065, 'bagging_fraction': 0.5383732940213594, 'lambda_l1': 4.6172226581130555, 'lambda_l2': 4.228239666885414, 'min_gain_to_split': 0.11765746086511997, 'min_child_weight': 2.4459990397622433}. Best is trial 24 with value: 2643.3637905333035.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1918]	valid_0's rmse: 2694.13


/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-09-25 11:51:05,670] Trial 29 finished with value: 2694.1336328860834 and parameters: {'learning_rate': 0.061797949861180757, 'num_leaves': 2444, 'max_depth': 11, 'min_data_in_leaf': 64, 'feature_fraction': 0.4638016088341487, 'bagging_fraction': 0.5409719765162476, 'lambda_l1': 4.817668945407842, 'lambda_l2': 3.7934097891791323, 'min_gain_to_split': 0.16450621717049108, 'min_child_weight': 2.3368013250940995}. Best is trial 24 with value: 2643.3637905333035.


Best trial: {'learning_rate': 0.017780312906297557, 'num_leaves': 1474, 'max_depth': 14, 'min_data_in_leaf': 86, 'feature_fraction': 0.45292978398075145, 'bagging_fraction': 0.4201674433320353, 'lambda_l1': 3.209550539996979, 'lambda_l2': 6.5289134117884915, 'min_gain_to_split': 0.10540894611143609, 'min_child_weight': 3.7742275788805566}


In [9]:
# Train the final LightGBM model with the best hyperparameters
best_params = study.best_trial.params

best_model = lgb.LGBMRegressor(**best_params, n_estimators=10000)

best_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
  #  early_stopping_rounds=50,
    categorical_feature=cat_feature_indices
)



LGBMRegressor(bagging_fraction=0.4201674433320353,
              feature_fraction=0.45292978398075145, lambda_l1=3.209550539996979,
              lambda_l2=6.5289134117884915, learning_rate=0.017780312906297557,
              max_depth=14, min_child_weight=3.7742275788805566,
              min_data_in_leaf=86, min_gain_to_split=0.10540894611143609,
              n_estimators=10000, num_leaves=1474)

In [ ]:
# Inspect the model's feature names (from the training phase)
model_features = best_model.feature_name_  # Or check best_model.booster_.feature_name() for lightgbm

# Print model's features and submission_set columns for debugging
print("Model features:", model_features)
print("Submission set columns:", submission_set.columns.tolist())

# Ensure the submission_set has the same features as the model
# Add missing columns with default values (e.g., 0 or the mean)
for feature in model_features:
    if feature not in submission_set.columns:
        submission_set[feature] = 0  # You can choose a more appropriate fill value


In [12]:
from sklearn.preprocessing import LabelEncoder

# List of categorical columns that need to be converted to numeric
categorical_columns = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline']

# Initialize LabelEncoder
label_encoders = {}

# Convert each categorical column to numeric using LabelEncoder
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    submission_set[col] = label_encoders[col].fit_transform(submission_set[col].astype(str))

# Print to check the converted types
print("Submission set dtypes after label encoding:", submission_set.dtypes)

# Ensure the submission_set has the same features as the model
submission_set_features = submission_set[model_features]

# Predict using the aligned features
submission_set['tow'] = best_model.predict(submission_set_features)

# Inspect the result
print(submission_set.head())


Submission set dtypes after label encoding: flight_id              int64
date                  object
callsign              object
adep                   int64
name_adep             object
                       ...  
Latitude_ades          int64
Longitude_ades         int64
elevation_gradient     int64
adep_geo_cluster       int64
ades_geo_cluster       int64
Length: 64, dtype: object
   flight_id        date                          callsign  adep  \
0  248753821  2022-01-01  3b3de0f3ad0ee192513995c02f7bf7cf   331   
1  248753822  2022-01-01  e06dd03d4a879ca37d9e18c1bd7cad16    11   
2  248754498  2022-01-01  2d3b1c962c78c4ebeef11bcd51b9e94c   158   
3  248757623  2022-01-01  81564432d3ee97c4bdf4cd8f006753dc    45   
4  248763603  2022-01-01  84be079d7e660db105d91f600b4b3d59    74   

                name_adep country_code_adep  ades        name_ades  \
0  Istanbul Sabiha Gokcen                TR   185             Lyon   
1                Brussels                BE   138     New York

In [13]:

from datetime import datetime
import pytz

saopaulo_tz = pytz.timezone('America/Sao_Paulo')

timestamp = datetime.now(saopaulo_tz).strftime('%Y%m%d_%H%M%S')


In [14]:

import os

# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")
submission_set.to_csv(submission_file, index=False)